In [2]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math
from typing import Tuple

Data source : https://github.com/python-engineer/pytorchTutorial/tree/master/data/wine

In [98]:
class WineDataset(Dataset):
    def __init__(self, transform=None) -> None:
        xy = np.loadtxt('../data/wine.csv', delimiter=',', dtype=np.float32, skiprows=1)
        self.x = xy[:,1:]
        self.y = xy[:,[0]] # ensure shape is (n x 1)
        self.n_samples = xy.shape[0]
        self.transform = transform

    def __getitem__(self, index) -> Tuple:
        sample = self.x[index], self.y[index]
        if self.transform:
            sample = self.transform(sample)
        return sample

    def __len__(self) -> int:
        return self.n_samples

class ToTensor: # a transformer
    def __call__(self, sample):
        inputs, labels = sample
        return torch.from_numpy(inputs), torch.from_numpy(labels)

class MulTransform:
    def __init__(self, factor) -> None:
        self.factor = factor
    
    def __call__(self, sample):
        inputs, labels = sample
        return self.factor * inputs, labels

composed = torchvision.transforms.Compose([ToTensor(), MulTransform(factor=2)])
dataset = WineDataset(transform=composed)
first_data = dataset[0]
features, labels = first_data
print(features, labels)


tensor([2.8460e+01, 3.4200e+00, 4.8600e+00, 3.1200e+01, 2.5400e+02, 5.6000e+00,
        6.1200e+00, 5.6000e-01, 4.5800e+00, 1.1280e+01, 2.0800e+00, 7.8400e+00,
        2.1300e+03]) tensor([1.])


In [99]:
n_epoch = 2
total_samples = len(dataset)
n_iterations = math.ceil(total_samples / 4)
print(total_samples, n_iterations)


178 45


In [100]:
n_features = dataset[0][0].shape[0]
n_features

13

In [139]:
class NueralNet2(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NueralNet2, self).__init__()
        self.linear1 = nn.Linear(in_features=input_size, out_features=hidden_size)
        self.relu = nn.ReLU() # activation function
        self.linear2 = nn.Linear(in_features=hidden_size, out_features=num_classes)

    def forward(self, X):
        out = self.linear1(X)
        out = self.relu(out)
        out = self.linear2(out)
        # no soft max at the end
        return out

model = NueralNet2(input_size=n_features, hidden_size=n_features*2, num_classes=3+1) # need to plus 1 for cross entropy loss
criterion = nn.CrossEntropyLoss() # applies softmax
lr = 0.01
alpha = 0.5
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr, weight_decay=alpha)
n_epoch = 100


In [140]:
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=2)

for epoch in range(n_epoch):
    total_loss = 0
    acc_tracker = []
    for i, (batch_X, batch_y) in enumerate(dataloader):
        y_pred = model(batch_X)
        batch_y = torch.squeeze(batch_y).type(torch.LongTensor)
        batch_loss = criterion(y_pred, batch_y)
        batch_loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        with torch.no_grad():
            total_loss += batch_loss.item()
            _, y_pred_cls = torch.max(y_pred, 1)
            acc = y_pred_cls.eq(batch_y).sum() / batch_y.shape[0]
            acc_tracker.append(acc)
    if (epoch+1) % 10 == 0:
        print(f'epoch {epoch+1}: Loss={total_loss:.3f}. Average Acc={np.mean(acc_tracker):.3f}')



epoch 10: Loss=56.286. Average Acc=0.661
epoch 20: Loss=38.906. Average Acc=0.594
epoch 30: Loss=35.893. Average Acc=0.644
epoch 40: Loss=32.190. Average Acc=0.617
epoch 50: Loss=34.932. Average Acc=0.617
epoch 60: Loss=35.153. Average Acc=0.628
epoch 70: Loss=35.721. Average Acc=0.622
epoch 80: Loss=33.066. Average Acc=0.633
epoch 90: Loss=35.115. Average Acc=0.622
epoch 100: Loss=34.380. Average Acc=0.633
